In [247]:
import hopsworks
from hsfs.client.exceptions import RestAPIError

# Establish connection to Hopsworks
project = hopsworks.login(
    api_key_value="O4IOxWozstKu0BFQ.07C1tbvgVI5C4XNLbLrGH4PS4t0EqBYN00ex8318TNIkl82WwDi3Vh9MidMrCA83"
)

# Access the Feature Store
fs = project.get_feature_store()

# List of feature groups and their versions
feature_groups = [
    {"name": "city_weather", "version": 1},
    {"name": "drivers_table", "version": 1},
    {"name": "routes_table", "version": 1},
    {"name": "routes_weather", "version": 1},
    {"name": "traffic_table", "version": 1},  # Corrected typo
    {"name": "truck_schedule_table", "version": 1},
    {"name": "trucks_table", "version": 1},
]
# Initialize an empty dictionary to hold DataFrames for each feature group
feature_group_data = {}

# Loop over the list of feature groups and download them
for fg in feature_groups:
    try:
        # Retrieve the feature group by its name and version
        feature_group = fs.get_feature_group(fg['name'], version=fg['version'])
        df = feature_group.read()  # Read the feature group as a DataFrame
        
        # Save the DataFrame in the dictionary with the feature group's name as the key
        feature_group_data[fg['name']] = df

        print(f"Downloaded feature group: {fg['name']} (version {fg['version']})")
        print(df.head())  # Optionally display the first few rows of the DataFrame

    except RestAPIError as e:
        print(f"Error downloading feature group: {fg['name']} (version {fg['version']})")
        print(e)

# After the loop, you will have each feature group stored in the `feature_group_data` dictionary
# Access each DataFrame like this: feature_group_data['city_weather'], etc.



Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1074326
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.63s) 
Downloaded feature group: city_weather (version 1)
   rowindex     city_id                      date  temp  wind_speed  \
0     45608  C-825b2625 2019-01-16 07:00:00+00:00    32           4   
1      4441  C-451776b7 2019-01-02 00:00:00+00:00    43           4   
2     18593  C-e5bfb4e5 2019-02-08 16:00:00+00:00     1          11   
3     47883  C-90e30162 2019-01-19 02:00:00+00:00    57          10   
4     44350  C-d3bb431c 2019-01-09 21:00:00+00:00    28           4   

                     description  precip  humidity  visibility  pressure  \
0            Light freezing rain     0.0        94           4      1025   
1                  Partly cloudy     0.0        29         

In [180]:
city_weather = feature_group_data['city_weather']
drivers_table = feature_group_data['drivers_table']
routes_table = feature_group_data['routes_table']
routes_weather = feature_group_data['routes_weather']
traffic_table = feature_group_data['traffic_table'] 
truck_schedule_table = feature_group_data['truck_schedule_table']
trucks_table = feature_group_data['trucks_table']


In [181]:
def assign_feature_group_data(feature_group_data, table_names):
    # Dynamically assign the DataFrame from feature_group_data to the variable names provided in table_names
    for table_name in table_names:
        globals()[table_name] = feature_group_data[table_name]
        print(f"Assigned {table_name} to variable.")
        
# List of table names to be assigned
table_names = ['city_weather', 'routes_weather', 'traffic_table', 'truck_schedule_table', 'trucks_table']

# Call the function to assign feature group data to variables
assign_feature_group_data(feature_group_data, table_names)

# Now you can access the variables dynamically assigned
print(truck_schedule_table.head())        

# 'drivers_table', 'routes_table'


Assigned city_weather to variable.
Assigned routes_weather to variable.
Assigned traffic_table to variable.
Assigned truck_schedule_table to variable.
Assigned trucks_table to variable.
   rowindex  truck_id    route_id       departure_date  \
0      8064  87511111  R-c0901e00  2019-02-09 07:00:00   
1      1515  18910102  R-8eabc8ee  2019-01-07 07:00:00   
2      5511  10946672  R-1ec2684e  2019-01-25 07:00:00   
3     10480  16497961  R-2e2bd615  2019-02-05 07:00:00   
4       212  18882093  R-1acdeb76  2019-01-01 07:00:00   

          estimated_arrival  delay                        eventtime  
0 2019-02-10 03:00:00+00:00      0 2024-09-30 23:03:26.588158+00:00  
1 2019-01-08 02:00:00+00:00      1 2024-09-30 23:03:26.588158+00:00  
2 2019-01-26 02:00:00+00:00      0 2024-09-30 23:03:26.588158+00:00  
3 2019-02-06 07:00:00+00:00      1 2024-09-30 23:03:26.588158+00:00  
4 2019-01-02 04:00:00+00:00      0 2024-09-30 23:03:26.588158+00:00  


In [182]:
# Create a dictionary of DataFrame names and DataFrames
dfs = {
    'city_weather': city_weather,
    'drivers_table': drivers_table,
    'routes_table': routes_table,
    'routes_weather': routes_weather,
    'traffic_table': traffic_table,
    'truck_schedule_table': truck_schedule_table,
    'trucks_table': trucks_table
}

            
# Updated function to drop multiple columns from the DataFrames
def drop_columns_from_dfs(dfs, columns):
    for df_name, df in dfs.items():
        columns_to_drop = [col for col in columns if col in df.columns]
        if columns_to_drop:
            df.drop(columns=columns_to_drop, inplace=True)
            print(f"Columns {columns_to_drop} dropped from {df_name}")
        else:
            print(f"No specified columns found in {df_name}")

# List of columns to drop
columns_to_remove = ['rowindex', 'event_time', 'eventtime']

# Call the function to drop the specified columns from all DataFrames
drop_columns_from_dfs(dfs, columns_to_remove)


Columns ['rowindex', 'eventtime'] dropped from city_weather
Columns ['rowindex', 'eventtime'] dropped from drivers_table
Columns ['rowindex', 'eventtime'] dropped from routes_table
Columns ['rowindex', 'eventtime'] dropped from routes_weather
Columns ['rowindex', 'eventtime'] dropped from traffic_table
Columns ['rowindex', 'eventtime'] dropped from truck_schedule_table
Columns ['rowindex', 'eventtime'] dropped from trucks_table


In [183]:
# Define the function to drop duplicate rows from multiple DataFrames
def drop_duplicates_from_dfs(dfs, columns_to_check):   
    for df_name, df in dfs.items():
        if df_name in columns_to_check:
            subset_columns = columns_to_check[df_name]
            before_dropping = df.shape[0]  # Count the number of rows before dropping duplicates
            df.drop_duplicates(subset=subset_columns, inplace=True)
            after_dropping = df.shape[0]  # Count the number of rows after dropping duplicates
            print(f"Dropped {before_dropping - after_dropping} duplicate rows from {df_name} based on columns: {subset_columns}")
        else:
            print(f"No duplicate check columns provided for {df_name}")
            
            
# Dictionary where the key is the DataFrame name, and the value is the subset of columns to check for duplicates
columns_to_check = {
    'city_weather': ['city_id', 'date'],
    'routes_weather': ['route_id', 'date'],
    'trucks_table': ['truck_id'],
    'drivers_table': ['driver_id'],
    'routes_table': ['route_id', 'destination_id', 'origin_id'],
    'truck_schedule': ['truck_id','route_id','departure_date']
}

# Call the function to drop duplicates from the DataFrames
drop_duplicates_from_dfs(dfs, columns_to_check)

# Check the result by printing the modified DataFrames
for df_name, df in dfs.items():
    print(f"\n{df_name} DataFrame after dropping duplicates:")
    print(df)            

Dropped 1027 duplicate rows from city_weather based on columns: ['city_id', 'date']
Dropped 0 duplicate rows from drivers_table based on columns: ['driver_id']
Dropped 0 duplicate rows from routes_table based on columns: ['route_id', 'destination_id', 'origin_id']
No duplicate check columns provided for routes_weather
No duplicate check columns provided for traffic_table
No duplicate check columns provided for truck_schedule_table
Dropped 0 duplicate rows from trucks_table based on columns: ['truck_id']

city_weather DataFrame after dropping duplicates:
          city_id                      date  temp  wind_speed  \
0      C-825b2625 2019-01-16 07:00:00+00:00    32           4   
1      C-451776b7 2019-01-02 00:00:00+00:00    43           4   
2      C-e5bfb4e5 2019-02-08 16:00:00+00:00     1          11   
3      C-90e30162 2019-01-19 02:00:00+00:00    57          10   
4      C-d3bb431c 2019-01-09 21:00:00+00:00    28           4   
...           ...                       ...   ... 

In [184]:
city_weather.head(5)

,city_id,date,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder
0,C-825b2625,2019-01-16 07:00:00+00:00,32,4,Light freezing rain,0.0,94,4,1025,0,0,0,0
1,C-451776b7,2019-01-02 00:00:00+00:00,43,4,Partly cloudy,0.0,29,6,1022,0,0,0,0
2,C-e5bfb4e5,2019-02-08 16:00:00+00:00,1,11,Overcast,0.0,86,6,1038,0,0,0,0
3,C-90e30162,2019-01-19 02:00:00+00:00,57,10,Moderate or heavy rain shower,0.1,94,5,1010,0,0,0,0
4,C-d3bb431c,2019-01-09 21:00:00+00:00,28,4,Heavy snow,0.0,97,1,1006,0,0,0,0


In [185]:
#code to drop the unnecessary columns in the city_weather dataframe
# List of columns to drop
columns_to_drop = ['chanceofrain', 'chanceoffog', 'chanceofsnow', 'chanceofthunder']

# Drop the specified columns from the city_weather DataFrame
city_weather.drop(columns=columns_to_drop, inplace=True)
routes_weather.drop(columns=columns_to_drop, inplace=True)

# Display the updated DataFrame
print(city_weather.head(5))
print(routes_weather.head(5))



      city_id                      date  temp  wind_speed  \
0  C-825b2625 2019-01-16 07:00:00+00:00    32           4   
1  C-451776b7 2019-01-02 00:00:00+00:00    43           4   
2  C-e5bfb4e5 2019-02-08 16:00:00+00:00     1          11   
3  C-90e30162 2019-01-19 02:00:00+00:00    57          10   
4  C-d3bb431c 2019-01-09 21:00:00+00:00    28           4   

                     description  precip  humidity  visibility  pressure  
0            Light freezing rain     0.0        94           4      1025  
1                  Partly cloudy     0.0        29           6      1022  
2                       Overcast     0.0        86           6      1038  
3  Moderate or heavy rain shower     0.1        94           5      1010  
4                     Heavy snow     0.0        97           1      1006  
     route_id                      date  temp  wind_speed  \
0  R-443c4074 2019-01-01 00:00:00+00:00    79           4   
1  R-26795240 2019-01-07 18:00:00+00:00    39          11   


In [186]:
# Rename the 'date' column to 'custom_date'
city_weather.rename(columns={'date': 'custom_date'}, inplace=True)

# Display the updated DataFrame
print(city_weather.head())

      city_id               custom_date  temp  wind_speed  \
0  C-825b2625 2019-01-16 07:00:00+00:00    32           4   
1  C-451776b7 2019-01-02 00:00:00+00:00    43           4   
2  C-e5bfb4e5 2019-02-08 16:00:00+00:00     1          11   
3  C-90e30162 2019-01-19 02:00:00+00:00    57          10   
4  C-d3bb431c 2019-01-09 21:00:00+00:00    28           4   

                     description  precip  humidity  visibility  pressure  
0            Light freezing rain     0.0        94           4      1025  
1                  Partly cloudy     0.0        29           6      1022  
2                       Overcast     0.0        86           6      1038  
3  Moderate or heavy rain shower     0.1        94           5      1010  
4                     Heavy snow     0.0        97           1      1006  


Feature Engineering

Merge Route_weather with Schedule data * create new data frame from scheduledf and add estimated_arrival and departure_date columns estimated_arriaval= df[estimated_arrival].dt.ceil("6H') departure_date= df[departure_date].dt.floor("6H")

In [187]:
truck_schedule_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12308 entries, 0 to 12307
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   truck_id           12308 non-null  int64              
 1   route_id           12308 non-null  object             
 2   departure_date     12308 non-null  object             
 3   estimated_arrival  12308 non-null  datetime64[us, UTC]
 4   delay              12308 non-null  int64              
dtypes: datetime64[us, UTC](1), int64(2), object(2)
memory usage: 480.9+ KB


In [188]:
routes_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396149 entries, 0 to 396148
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   route_id     396149 non-null  object             
 1   date         396149 non-null  datetime64[us, UTC]
 2   temp         396149 non-null  int64              
 3   wind_speed   396149 non-null  int64              
 4   description  396149 non-null  object             
 5   precip       396149 non-null  float64            
 6   humidity     396149 non-null  int64              
 7   visibility   396149 non-null  int64              
 8   pressure     396149 non-null  int64              
dtypes: datetime64[us, UTC](1), float64(1), int64(5), object(2)
memory usage: 27.2+ MB


    * Merge the resulant data frame with route_weather on route_id and date (left)

In [189]:
import pandas as pd

def process_dates(df, estimated_arrival_column, departure_date_column):
    # Function to convert to UTC if not already timezone-aware
    def convert_to_utc(series):
        if series.dt.tz is None:  # Check if the series is not timezone-aware
            return series.dt.tz_localize('UTC')
        else:
            return series.dt.tz_convert('UTC')  # Convert to UTC if it already has a timezone

    # Step 1: Create a copy of the DataFrame to avoid modifying the original
    df_copy = df.copy()

    # Step 2: Convert both columns to datetime format and set to UTC timezone
    df_copy[estimated_arrival_column] = convert_to_utc(pd.to_datetime(df_copy[estimated_arrival_column], errors='coerce'))
    df_copy[departure_date_column] = convert_to_utc(pd.to_datetime(df_copy[departure_date_column], errors='coerce'))

    # Step 3: Apply ceil operation to estimated arrival
    df_copy[estimated_arrival_column] = df_copy[estimated_arrival_column].dt.ceil("6H")

    # Step 4: Floor the 'departure_date' column to the nearest 6 hours
    df_copy[departure_date_column] = df_copy[departure_date_column].dt.floor("6H")

    # Step 5: Create a new column 'date' with date ranges between 'departure_date' and 'estimated_arrival'
    df_copy['date'] = [
        pd.date_range(start=row[departure_date_column], end=row[estimated_arrival_column], freq='6H')
        for index, row in df_copy.iterrows()
    ]
    
    # Step 6: Explode the 'date' column to separate rows for each date in the range
    df_copy = df_copy.explode('date').reset_index(drop=True)

    return df_copy


# Define target tables
target_tables = ['truck_schedule_table']
departure_date_column = 'departure_date'
estimated_arrival_column = 'estimated_arrival'

# Process the DataFrames
for table_name in list(dfs.keys()):  # Use list(dfs.keys()) to avoid changing the dictionary during iteration
    df = dfs[table_name]
    if table_name in target_tables:
        # Create a copy and process dates
        result_df_copy = process_dates(df, estimated_arrival_column, departure_date_column)
        
        # Merge with route_weather on 'route_id' and 'date'
        scheduled_weather = pd.merge(result_df_copy, routes_weather, on=['route_id', 'date'], how='left')

        # Display the merged DataFrame
        print(scheduled_weather)

        # Save the processed DataFrame under a new key
        dfs[f'{table_name}_processed'] = result_df_copy


       truck_id    route_id            departure_date  \
0      87511111  R-c0901e00 2019-02-09 06:00:00+00:00   
1      87511111  R-c0901e00 2019-02-09 06:00:00+00:00   
2      87511111  R-c0901e00 2019-02-09 06:00:00+00:00   
3      87511111  R-c0901e00 2019-02-09 06:00:00+00:00   
4      87511111  R-c0901e00 2019-02-09 06:00:00+00:00   
...         ...         ...                       ...   
69310  64713507  R-42c1d264 2019-01-09 06:00:00+00:00   
69311  64713507  R-42c1d264 2019-01-09 06:00:00+00:00   
69312  64713507  R-42c1d264 2019-01-09 06:00:00+00:00   
69313  64713507  R-42c1d264 2019-01-09 06:00:00+00:00   
69314  64713507  R-42c1d264 2019-01-09 06:00:00+00:00   

              estimated_arrival  delay                      date  temp  \
0     2019-02-10 06:00:00+00:00      0 2019-02-09 06:00:00+00:00  63.0   
1     2019-02-10 06:00:00+00:00      0 2019-02-09 12:00:00+00:00  52.0   
2     2019-02-10 06:00:00+00:00      0 2019-02-09 18:00:00+00:00  52.0   
3     2019-02-10 06

In [190]:
scheduled_weather.head()

,truck_id,route_id,departure_date,estimated_arrival,delay,date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,87511111,R-c0901e00,2019-02-09 06:00:00+00:00,2019-02-10 06:00:00+00:00,0,2019-02-09 06:00:00+00:00,63.0,6.0,Patchy rain possible,0.0,82.0,6.0,1011.0
1,87511111,R-c0901e00,2019-02-09 06:00:00+00:00,2019-02-10 06:00:00+00:00,0,2019-02-09 12:00:00+00:00,52.0,2.0,Clear,0.0,92.0,4.0,1021.0
2,87511111,R-c0901e00,2019-02-09 06:00:00+00:00,2019-02-10 06:00:00+00:00,0,2019-02-09 18:00:00+00:00,52.0,2.0,Mist,0.0,93.0,3.0,1021.0
3,87511111,R-c0901e00,2019-02-09 06:00:00+00:00,2019-02-10 06:00:00+00:00,0,2019-02-10 00:00:00+00:00,63.0,6.0,Patchy rain possible,0.0,84.0,6.0,1011.0
4,87511111,R-c0901e00,2019-02-09 06:00:00+00:00,2019-02-10 06:00:00+00:00,0,2019-02-10 06:00:00+00:00,63.0,6.0,Light rain shower,0.0,87.0,6.0,1011.0


In [191]:
import numpy as np
# Define a custom function to calculate mode with error handling
def custom_mode(x):
    mode_values = x.mode()
    if len(mode_values) > 0:
        return mode_values.iloc[0]
    else:
        return np.nan  # or a default value, such as '' or np.nan

# Group by specified columns and aggregate
scheduled_weather_grp = scheduled_weather.groupby(['truck_id','route_id'], as_index=False).agg(
             route_avg_temp=('temp','mean'),
             route_avg_wind_speed=('wind_speed','mean'),
             route_avg_precip=('precip','mean'),
             route_avg_humidity=('humidity','mean'),
             route_avg_visibility=('visibility','mean'),
             route_avg_pressure=('pressure','mean'),
             route_description=('description', custom_mode)
         )

scheduled_weather_grp

,truck_id,route_id,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,route_avg_pressure,route_description
0,10008392,R-06dfe39e,49.000000,5.818182,0.00,80.909091,5.818182,1015.181818,Partly cloudy
1,10008392,R-35dca729,56.600000,9.400000,0.00,72.200000,5.700000,1013.300000,Partly cloudy
2,10008392,R-51d1a32a,46.636364,8.909091,0.00,76.181818,6.000000,1016.818182,Partly cloudy
3,10008392,R-7ab5ae85,58.750000,7.000000,0.00,82.250000,4.000000,1016.000000,Sunny
4,10008392,R-c6f8e036,48.181818,6.545455,0.00,87.727273,5.545455,1011.727273,Clear
...,...,...,...,...,...,...,...,...,...
12235,99981667,R-abbf83d1,59.000000,7.500000,0.00,80.750000,5.500000,1014.500000,Light rain shower
12236,99981667,R-e9bf94ff,56.500000,3.250000,0.00,76.500000,4.000000,1015.500000,Mist
12237,99981667,R-ed1094fd,64.500000,10.500000,0.00,42.500000,6.000000,1016.750000,Clear
12238,99981667,R-f363ad37,79.000000,11.800000,0.02,84.800000,4.800000,1013.600000,Clear


Merge schedule df with schedule_weather_grp df

In [192]:
schedule_weather_merge=truck_schedule_table.merge(scheduled_weather_grp,on=['truck_id','route_id'],how='left')
schedule_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,route_avg_pressure,route_description
0,87511111,R-c0901e00,2019-02-09 07:00:00,2019-02-10 03:00:00+00:00,0,58.600000,4.400000,0.0,87.600000,5.000000,1015.000000,Patchy rain possible
1,18910102,R-8eabc8ee,2019-01-07 07:00:00,2019-01-08 02:00:00+00:00,1,46.000000,8.000000,0.0,97.333333,4.333333,1010.000000,Overcast
2,10946672,R-1ec2684e,2019-01-25 07:00:00,2019-01-26 02:00:00+00:00,0,74.000000,4.600000,0.0,43.800000,6.000000,1015.000000,Sunny
3,16497961,R-2e2bd615,2019-02-05 07:00:00,2019-02-06 07:00:00+00:00,1,60.000000,4.666667,0.0,94.000000,2.666667,1014.500000,Mist
4,18882093,R-1acdeb76,2019-01-01 07:00:00,2019-01-02 04:00:00+00:00,0,48.400000,3.600000,0.0,70.200000,6.000000,1017.200000,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...
12303,95354743,R-6d4858a1,2019-01-07 07:00:00,2019-01-08 00:00:00+00:00,0,55.000000,5.000000,0.1,96.500000,4.000000,1020.000000,Moderate rain
12304,57362582,R-1d7b758f,2019-01-10 07:00:00,2019-01-10 08:00:00+00:00,0,65.000000,8.000000,0.0,82.000000,6.000000,1009.000000,Clear
12305,22376424,R-c165d4fd,2019-02-06 07:00:00,2019-02-06 08:00:00+00:00,0,59.000000,8.000000,0.0,64.500000,6.000000,1021.000000,Clear
12306,10234289,R-e3a6b473,2019-02-12 07:00:00,2019-02-12 21:00:00+00:00,0,59.750000,5.000000,0.0,84.250000,6.000000,1010.000000,Clear


 * Find Origin and Destination city Weather

In [193]:
#take hourly as weather data available hourly

import pandas as pd

# Create a copy of the truck_schedule_table DataFrame
nearest_hour_schedule_df = truck_schedule_table.copy()

# Convert 'estimated_arrival' and 'departure_date' columns to datetime (if they aren't already)
nearest_hour_schedule_df['estimated_arrival'] = pd.to_datetime(nearest_hour_schedule_df['estimated_arrival'], errors='coerce')
nearest_hour_schedule_df['departure_date'] = pd.to_datetime(nearest_hour_schedule_df['departure_date'], errors='coerce')

# Round 'estimated_arrival' to the nearest hour
nearest_hour_schedule_df['estimated_arrival_nearest_hour'] = nearest_hour_schedule_df['estimated_arrival'].dt.round("H")

# Round 'departure_date' to the nearest hour
nearest_hour_schedule_df['departure_date_nearest_hour'] = nearest_hour_schedule_df['departure_date'].dt.round("H")

# Merge the result with routes_table on 'route_id'
nearest_hour_schedule_route_df = pd.merge(nearest_hour_schedule_df, routes_table, on='route_id', how='left')

# Display the resulting DataFrame
print(nearest_hour_schedule_route_df)


       truck_id    route_id      departure_date         estimated_arrival  \
0      87511111  R-c0901e00 2019-02-09 07:00:00 2019-02-10 03:00:00+00:00   
1      18910102  R-8eabc8ee 2019-01-07 07:00:00 2019-01-08 02:00:00+00:00   
2      10946672  R-1ec2684e 2019-01-25 07:00:00 2019-01-26 02:00:00+00:00   
3      16497961  R-2e2bd615 2019-02-05 07:00:00 2019-02-06 07:00:00+00:00   
4      18882093  R-1acdeb76 2019-01-01 07:00:00 2019-01-02 04:00:00+00:00   
...         ...         ...                 ...                       ...   
12303  95354743  R-6d4858a1 2019-01-07 07:00:00 2019-01-08 00:00:00+00:00   
12304  57362582  R-1d7b758f 2019-01-10 07:00:00 2019-01-10 08:00:00+00:00   
12305  22376424  R-c165d4fd 2019-02-06 07:00:00 2019-02-06 08:00:00+00:00   
12306  10234289  R-e3a6b473 2019-02-12 07:00:00 2019-02-12 21:00:00+00:00   
12307  64713507  R-42c1d264 2019-01-09 07:00:00 2019-01-13 09:00:00+00:00   

       delay estimated_arrival_nearest_hour departure_date_nearest_hour  \


In [194]:
# Assume 'city_weather' is your original DataFrame
origin_weather_data= city_weather.copy()  # First copy
destination_weather_data = city_weather.copy()  # Second copy

In [195]:
nearest_hour_schedule_route_df.head(5)

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours
0,87511111,R-c0901e00,2019-02-09 07:00:00,2019-02-10 03:00:00+00:00,0,2019-02-10 03:00:00+00:00,2019-02-09 07:00:00,C-9bbf5d8a,C-2c9e75ef,1016.34,20.33
1,18910102,R-8eabc8ee,2019-01-07 07:00:00,2019-01-08 02:00:00+00:00,1,2019-01-08 02:00:00+00:00,2019-01-07 07:00:00,C-6ea51d66,C-7212cebe,950.00,19.00
2,10946672,R-1ec2684e,2019-01-25 07:00:00,2019-01-26 02:00:00+00:00,0,2019-01-26 02:00:00+00:00,2019-01-25 07:00:00,C-2bd47dc5,C-90e30162,952.76,19.06
3,16497961,R-2e2bd615,2019-02-05 07:00:00,2019-02-06 07:00:00+00:00,1,2019-02-06 07:00:00+00:00,2019-02-05 07:00:00,C-841ebdcb,C-5d86b887,1212.33,24.25
4,18882093,R-1acdeb76,2019-01-01 07:00:00,2019-01-02 04:00:00+00:00,0,2019-01-02 04:00:00+00:00,2019-01-01 07:00:00,C-84f378bb,C-c7cacd1d,1077.52,21.55


In [196]:
origin_weather_data

,city_id,custom_date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,C-825b2625,2019-01-16 07:00:00+00:00,32,4,Light freezing rain,0.0,94,4,1025
1,C-451776b7,2019-01-02 00:00:00+00:00,43,4,Partly cloudy,0.0,29,6,1022
2,C-e5bfb4e5,2019-02-08 16:00:00+00:00,1,11,Overcast,0.0,86,6,1038
3,C-90e30162,2019-01-19 02:00:00+00:00,57,10,Moderate or heavy rain shower,0.1,94,5,1010
4,C-d3bb431c,2019-01-09 21:00:00+00:00,28,4,Heavy snow,0.0,97,1,1006
...,...,...,...,...,...,...,...,...,...
51829,C-19236709,2019-01-21 01:00:00+00:00,5,6,Partly cloudy,0.0,91,6,1034
51830,C-2c9e75ef,2019-02-11 13:00:00+00:00,52,14,Light rain,0.0,81,6,1007
51831,C-451776b7,2019-01-09 01:00:00+00:00,63,2,Clear,0.0,70,6,1020
51832,C-ef47bdcd,2019-01-28 19:00:00+00:00,18,2,Patchy heavy snow,0.0,82,4,1028


* reate a copy of the 'weather_df' DataFrame for manipulation
            * Drop the 'date' and 'hour' columns from 'origin_weather_data'
            * Create a copy of the 'weather_df' DataFrame for manipulation
            * Drop the 'date' and 'hour' columns from 'destination_weather_data'
            * Merge 'nearest_hour_schedule_route_df' with 'origin_weather_data' based on specified columns
            * Merge 'origin_weather_merge' with 'destination_weather_data' based on specified columns

so basically here I already dropped hour column by merging it into date column and later renamed it to customdate column. Hence nothing left here to drop            

In [197]:
#Merge 'nearest_hour_schedule_route_df' with 'origin_weather_data' based on specified columns

# Ensure both DataFrames have the datetime columns in UTC format

# Check if departure_date_nearest_hour is timezone-naive
if nearest_hour_schedule_route_df['departure_date_nearest_hour'].dt.tz is None:
    # Localize to UTC if naive
    nearest_hour_schedule_route_df['departure_date_nearest_hour'] = nearest_hour_schedule_route_df['departure_date_nearest_hour'].dt.tz_localize('UTC')

# Check if custom_date is timezone-naive
if origin_weather_data['custom_date'].dt.tz is None:
    # Localize to UTC if naive
    origin_weather_data['custom_date'] = pd.to_datetime(origin_weather_data['custom_date'], errors='coerce').dt.tz_localize('UTC')

# Specify the columns to merge on
left_merge_columns = ['origin_id', 'departure_date_nearest_hour']  # Columns from nearest_hour_schedule_route_df
right_merge_columns = ['city_id', 'custom_date']  # Columns from origin_weather_data

# Merge the DataFrames
origin_weather_merge = pd.merge(
    nearest_hour_schedule_route_df, 
    origin_weather_data, 
    left_on=left_merge_columns, 
    right_on=right_merge_columns, 
    how='left'
)

# Display the resulting merged DataFrame
origin_weather_merge


,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours,city_id,custom_date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,87511111,R-c0901e00,2019-02-09 07:00:00,2019-02-10 03:00:00+00:00,0,2019-02-10 03:00:00+00:00,2019-02-09 07:00:00+00:00,C-9bbf5d8a,C-2c9e75ef,1016.34,20.33,C-9bbf5d8a,2019-02-09 07:00:00+00:00,73.0,3.0,Sunny,0.0,65.0,6.0,1014.0
1,18910102,R-8eabc8ee,2019-01-07 07:00:00,2019-01-08 02:00:00+00:00,1,2019-01-08 02:00:00+00:00,2019-01-07 07:00:00+00:00,C-6ea51d66,C-7212cebe,950.00,19.00,C-6ea51d66,2019-01-07 07:00:00+00:00,27.0,6.0,Partly cloudy,0.0,40.0,6.0,1032.0
2,10946672,R-1ec2684e,2019-01-25 07:00:00,2019-01-26 02:00:00+00:00,0,2019-01-26 02:00:00+00:00,2019-01-25 07:00:00+00:00,C-2bd47dc5,C-90e30162,952.76,19.06,C-2bd47dc5,2019-01-25 07:00:00+00:00,32.0,9.0,Sunny,0.0,59.0,6.0,1015.0
3,16497961,R-2e2bd615,2019-02-05 07:00:00,2019-02-06 07:00:00+00:00,1,2019-02-06 07:00:00+00:00,2019-02-05 07:00:00+00:00,C-841ebdcb,C-5d86b887,1212.33,24.25,C-841ebdcb,2019-02-05 07:00:00+00:00,66.0,11.0,Fog,0.0,96.0,1.0,1015.0
4,18882093,R-1acdeb76,2019-01-01 07:00:00,2019-01-02 04:00:00+00:00,0,2019-01-02 04:00:00+00:00,2019-01-01 07:00:00+00:00,C-84f378bb,C-c7cacd1d,1077.52,21.55,C-84f378bb,2019-01-01 07:00:00+00:00,21.0,15.0,Partly cloudy,0.0,58.0,6.0,1032.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12303,95354743,R-6d4858a1,2019-01-07 07:00:00,2019-01-08 00:00:00+00:00,0,2019-01-08 00:00:00+00:00,2019-01-07 07:00:00+00:00,C-2bd47dc5,C-825b2625,860.48,17.21,C-2bd47dc5,2019-01-07 07:00:00+00:00,28.0,5.0,Sunny,0.0,38.0,6.0,1032.0
12304,57362582,R-1d7b758f,2019-01-10 07:00:00,2019-01-10 08:00:00+00:00,0,2019-01-10 08:00:00+00:00,2019-01-10 07:00:00+00:00,C-b5ff31cd,C-40a81eb1,53.19,1.06,C-b5ff31cd,2019-01-10 07:00:00+00:00,28.0,14.0,Moderate snow,0.0,68.0,4.0,1000.0
12305,22376424,R-c165d4fd,2019-02-06 07:00:00,2019-02-06 08:00:00+00:00,0,2019-02-06 08:00:00+00:00,2019-02-06 07:00:00+00:00,C-c4565ee8,C-328bd8d3,50.02,1.00,C-c4565ee8,2019-02-06 07:00:00+00:00,50.0,8.0,Moderate rain,0.0,98.0,4.0,1013.0
12306,10234289,R-e3a6b473,2019-02-12 07:00:00,2019-02-12 21:00:00+00:00,0,2019-02-12 21:00:00+00:00,2019-02-12 07:00:00+00:00,C-c7cacd1d,C-b6e04c88,705.50,14.11,C-c7cacd1d,2019-02-12 07:00:00+00:00,25.0,12.0,Overcast,0.0,92.0,4.0,1021.0


In [198]:
destination_weather_data

,city_id,custom_date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,C-825b2625,2019-01-16 07:00:00+00:00,32,4,Light freezing rain,0.0,94,4,1025
1,C-451776b7,2019-01-02 00:00:00+00:00,43,4,Partly cloudy,0.0,29,6,1022
2,C-e5bfb4e5,2019-02-08 16:00:00+00:00,1,11,Overcast,0.0,86,6,1038
3,C-90e30162,2019-01-19 02:00:00+00:00,57,10,Moderate or heavy rain shower,0.1,94,5,1010
4,C-d3bb431c,2019-01-09 21:00:00+00:00,28,4,Heavy snow,0.0,97,1,1006
...,...,...,...,...,...,...,...,...,...
51829,C-19236709,2019-01-21 01:00:00+00:00,5,6,Partly cloudy,0.0,91,6,1034
51830,C-2c9e75ef,2019-02-11 13:00:00+00:00,52,14,Light rain,0.0,81,6,1007
51831,C-451776b7,2019-01-09 01:00:00+00:00,63,2,Clear,0.0,70,6,1020
51832,C-ef47bdcd,2019-01-28 19:00:00+00:00,18,2,Patchy heavy snow,0.0,82,4,1028


* Merge 'origin_weather_merge' with 'destination_weather_data' based on specified columns

In [200]:
#Merge 'origin_weather_merge' with 'destination_weather_data' based on specified columns

# Ensure both DataFrames have the datetime columns in UTC format

# Check if estimated_arrival_nearest_hour is timezone-naive
if origin_weather_merge['estimated_arrival_nearest_hour'].dt.tz is None:
    # Localize to UTC if naive
    origin_weather_merge['estimated_arrival_nearest_hour'] = origin_weather_merge['estimated_arrival_nearest_hour'].dt.tz_localize('UTC')

# Check if custom_date is timezone-naive
if destination_weather_data['custom_date'].dt.tz is None:
    # Localize to UTC if naive
    destination_weather_data['custom_date'] = pd.to_datetime(destination_weather_data['custom_date'], errors='coerce').dt.tz_localize('UTC')

# Specify the columns to merge on
left_merge_columns = ['destination_id', 'estimated_arrival_nearest_hour']  # Columns from origin_weather_merge
right_merge_columns = ['city_id', 'custom_date']  # Columns from destination_weather_data

# Merge the DataFrames
schedule_data_merge = pd.merge(
    origin_weather_merge, 
    destination_weather_data, 
    left_on=left_merge_columns, 
    right_on=right_merge_columns, 
    how='left'
)

# Display the resulting merged DataFrame
schedule_data_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,pressure_x,city_id_y,custom_date_y,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y
0,87511111,R-c0901e00,2019-02-09 07:00:00,2019-02-10 03:00:00+00:00,0,2019-02-10 03:00:00+00:00,2019-02-09 07:00:00+00:00,C-9bbf5d8a,C-2c9e75ef,1016.34,...,1014.0,C-2c9e75ef,2019-02-10 03:00:00+00:00,34.0,6.0,Overcast,0.0,89.0,6.0,1025.0
1,18910102,R-8eabc8ee,2019-01-07 07:00:00,2019-01-08 02:00:00+00:00,1,2019-01-08 02:00:00+00:00,2019-01-07 07:00:00+00:00,C-6ea51d66,C-7212cebe,950.00,...,1032.0,C-7212cebe,2019-01-08 02:00:00+00:00,34.0,5.0,Mist,0.0,95.0,1.0,1008.0
2,10946672,R-1ec2684e,2019-01-25 07:00:00,2019-01-26 02:00:00+00:00,0,2019-01-26 02:00:00+00:00,2019-01-25 07:00:00+00:00,C-2bd47dc5,C-90e30162,952.76,...,1015.0,C-90e30162,2019-01-26 02:00:00+00:00,34.0,2.0,Partly cloudy,0.0,75.0,6.0,1027.0
3,16497961,R-2e2bd615,2019-02-05 07:00:00,2019-02-06 07:00:00+00:00,1,2019-02-06 07:00:00+00:00,2019-02-05 07:00:00+00:00,C-841ebdcb,C-5d86b887,1212.33,...,1015.0,C-5d86b887,2019-02-06 07:00:00+00:00,48.0,4.0,Mist,0.0,89.0,3.0,1022.0
4,18882093,R-1acdeb76,2019-01-01 07:00:00,2019-01-02 04:00:00+00:00,0,2019-01-02 04:00:00+00:00,2019-01-01 07:00:00+00:00,C-84f378bb,C-c7cacd1d,1077.52,...,1032.0,C-c7cacd1d,2019-01-02 04:00:00+00:00,19.0,5.0,Partly cloudy,0.0,77.0,6.0,1027.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12303,95354743,R-6d4858a1,2019-01-07 07:00:00,2019-01-08 00:00:00+00:00,0,2019-01-08 00:00:00+00:00,2019-01-07 07:00:00+00:00,C-2bd47dc5,C-825b2625,860.48,...,1032.0,C-825b2625,2019-01-08 00:00:00+00:00,46.0,6.0,Clear,0.0,79.0,6.0,1011.0
12304,57362582,R-1d7b758f,2019-01-10 07:00:00,2019-01-10 08:00:00+00:00,0,2019-01-10 08:00:00+00:00,2019-01-10 07:00:00+00:00,C-b5ff31cd,C-40a81eb1,53.19,...,1000.0,C-40a81eb1,2019-01-10 08:00:00+00:00,34.0,9.0,Light snow,0.0,60.0,6.0,1000.0
12305,22376424,R-c165d4fd,2019-02-06 07:00:00,2019-02-06 08:00:00+00:00,0,2019-02-06 08:00:00+00:00,2019-02-06 07:00:00+00:00,C-c4565ee8,C-328bd8d3,50.02,...,1013.0,C-328bd8d3,2019-02-06 08:00:00+00:00,50.0,9.0,Moderate rain,0.1,98.0,4.0,1012.0
12306,10234289,R-e3a6b473,2019-02-12 07:00:00,2019-02-12 21:00:00+00:00,0,2019-02-12 21:00:00+00:00,2019-02-12 07:00:00+00:00,C-c7cacd1d,C-b6e04c88,705.50,...,1021.0,C-b6e04c88,2019-02-12 21:00:00+00:00,28.0,16.0,Clear,0.0,61.0,6.0,1020.0


Traffic and Schedule Data Merge

In [202]:
# Create a copy of the schedule_data_merge DataFrame
schedule_data_copy = schedule_data_merge.copy()
schedule_data_copy

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,pressure_x,city_id_y,custom_date_y,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y
0,87511111,R-c0901e00,2019-02-09 07:00:00,2019-02-10 03:00:00+00:00,0,2019-02-10 03:00:00+00:00,2019-02-09 07:00:00+00:00,C-9bbf5d8a,C-2c9e75ef,1016.34,...,1014.0,C-2c9e75ef,2019-02-10 03:00:00+00:00,34.0,6.0,Overcast,0.0,89.0,6.0,1025.0
1,18910102,R-8eabc8ee,2019-01-07 07:00:00,2019-01-08 02:00:00+00:00,1,2019-01-08 02:00:00+00:00,2019-01-07 07:00:00+00:00,C-6ea51d66,C-7212cebe,950.00,...,1032.0,C-7212cebe,2019-01-08 02:00:00+00:00,34.0,5.0,Mist,0.0,95.0,1.0,1008.0
2,10946672,R-1ec2684e,2019-01-25 07:00:00,2019-01-26 02:00:00+00:00,0,2019-01-26 02:00:00+00:00,2019-01-25 07:00:00+00:00,C-2bd47dc5,C-90e30162,952.76,...,1015.0,C-90e30162,2019-01-26 02:00:00+00:00,34.0,2.0,Partly cloudy,0.0,75.0,6.0,1027.0
3,16497961,R-2e2bd615,2019-02-05 07:00:00,2019-02-06 07:00:00+00:00,1,2019-02-06 07:00:00+00:00,2019-02-05 07:00:00+00:00,C-841ebdcb,C-5d86b887,1212.33,...,1015.0,C-5d86b887,2019-02-06 07:00:00+00:00,48.0,4.0,Mist,0.0,89.0,3.0,1022.0
4,18882093,R-1acdeb76,2019-01-01 07:00:00,2019-01-02 04:00:00+00:00,0,2019-01-02 04:00:00+00:00,2019-01-01 07:00:00+00:00,C-84f378bb,C-c7cacd1d,1077.52,...,1032.0,C-c7cacd1d,2019-01-02 04:00:00+00:00,19.0,5.0,Partly cloudy,0.0,77.0,6.0,1027.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12303,95354743,R-6d4858a1,2019-01-07 07:00:00,2019-01-08 00:00:00+00:00,0,2019-01-08 00:00:00+00:00,2019-01-07 07:00:00+00:00,C-2bd47dc5,C-825b2625,860.48,...,1032.0,C-825b2625,2019-01-08 00:00:00+00:00,46.0,6.0,Clear,0.0,79.0,6.0,1011.0
12304,57362582,R-1d7b758f,2019-01-10 07:00:00,2019-01-10 08:00:00+00:00,0,2019-01-10 08:00:00+00:00,2019-01-10 07:00:00+00:00,C-b5ff31cd,C-40a81eb1,53.19,...,1000.0,C-40a81eb1,2019-01-10 08:00:00+00:00,34.0,9.0,Light snow,0.0,60.0,6.0,1000.0
12305,22376424,R-c165d4fd,2019-02-06 07:00:00,2019-02-06 08:00:00+00:00,0,2019-02-06 08:00:00+00:00,2019-02-06 07:00:00+00:00,C-c4565ee8,C-328bd8d3,50.02,...,1013.0,C-328bd8d3,2019-02-06 08:00:00+00:00,50.0,9.0,Moderate rain,0.1,98.0,4.0,1012.0
12306,10234289,R-e3a6b473,2019-02-12 07:00:00,2019-02-12 21:00:00+00:00,0,2019-02-12 21:00:00+00:00,2019-02-12 07:00:00+00:00,C-c7cacd1d,C-b6e04c88,705.50,...,1021.0,C-b6e04c88,2019-02-12 21:00:00+00:00,28.0,16.0,Clear,0.0,61.0,6.0,1020.0


* Round 'estimated_arrival' times to the nearest hour
* Round 'departure_date' times to the nearest hour

In [204]:
# Round 'estimated_arrival' to the nearest hour
schedule_data_copy['estimated_arrival'] = schedule_data_copy['estimated_arrival'].dt.round('H')

# Round 'departure_date' to the nearest hour
schedule_data_copy['departure_date'] = schedule_data_copy['departure_date'].dt.round('H')

# Display the modified DataFrame
print(schedule_data_copy[['estimated_arrival', 'departure_date']].head())

nearest_hour_schedule_df = schedule_data_copy
nearest_hour_schedule_df


          estimated_arrival      departure_date
0 2019-02-10 03:00:00+00:00 2019-02-09 07:00:00
1 2019-01-08 02:00:00+00:00 2019-01-07 07:00:00
2 2019-01-26 02:00:00+00:00 2019-01-25 07:00:00
3 2019-02-06 07:00:00+00:00 2019-02-05 07:00:00
4 2019-01-02 04:00:00+00:00 2019-01-01 07:00:00


,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,pressure_x,city_id_y,custom_date_y,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y
0,87511111,R-c0901e00,2019-02-09 07:00:00,2019-02-10 03:00:00+00:00,0,2019-02-10 03:00:00+00:00,2019-02-09 07:00:00+00:00,C-9bbf5d8a,C-2c9e75ef,1016.34,...,1014.0,C-2c9e75ef,2019-02-10 03:00:00+00:00,34.0,6.0,Overcast,0.0,89.0,6.0,1025.0
1,18910102,R-8eabc8ee,2019-01-07 07:00:00,2019-01-08 02:00:00+00:00,1,2019-01-08 02:00:00+00:00,2019-01-07 07:00:00+00:00,C-6ea51d66,C-7212cebe,950.00,...,1032.0,C-7212cebe,2019-01-08 02:00:00+00:00,34.0,5.0,Mist,0.0,95.0,1.0,1008.0
2,10946672,R-1ec2684e,2019-01-25 07:00:00,2019-01-26 02:00:00+00:00,0,2019-01-26 02:00:00+00:00,2019-01-25 07:00:00+00:00,C-2bd47dc5,C-90e30162,952.76,...,1015.0,C-90e30162,2019-01-26 02:00:00+00:00,34.0,2.0,Partly cloudy,0.0,75.0,6.0,1027.0
3,16497961,R-2e2bd615,2019-02-05 07:00:00,2019-02-06 07:00:00+00:00,1,2019-02-06 07:00:00+00:00,2019-02-05 07:00:00+00:00,C-841ebdcb,C-5d86b887,1212.33,...,1015.0,C-5d86b887,2019-02-06 07:00:00+00:00,48.0,4.0,Mist,0.0,89.0,3.0,1022.0
4,18882093,R-1acdeb76,2019-01-01 07:00:00,2019-01-02 04:00:00+00:00,0,2019-01-02 04:00:00+00:00,2019-01-01 07:00:00+00:00,C-84f378bb,C-c7cacd1d,1077.52,...,1032.0,C-c7cacd1d,2019-01-02 04:00:00+00:00,19.0,5.0,Partly cloudy,0.0,77.0,6.0,1027.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12303,95354743,R-6d4858a1,2019-01-07 07:00:00,2019-01-08 00:00:00+00:00,0,2019-01-08 00:00:00+00:00,2019-01-07 07:00:00+00:00,C-2bd47dc5,C-825b2625,860.48,...,1032.0,C-825b2625,2019-01-08 00:00:00+00:00,46.0,6.0,Clear,0.0,79.0,6.0,1011.0
12304,57362582,R-1d7b758f,2019-01-10 07:00:00,2019-01-10 08:00:00+00:00,0,2019-01-10 08:00:00+00:00,2019-01-10 07:00:00+00:00,C-b5ff31cd,C-40a81eb1,53.19,...,1000.0,C-40a81eb1,2019-01-10 08:00:00+00:00,34.0,9.0,Light snow,0.0,60.0,6.0,1000.0
12305,22376424,R-c165d4fd,2019-02-06 07:00:00,2019-02-06 08:00:00+00:00,0,2019-02-06 08:00:00+00:00,2019-02-06 07:00:00+00:00,C-c4565ee8,C-328bd8d3,50.02,...,1013.0,C-328bd8d3,2019-02-06 08:00:00+00:00,50.0,9.0,Moderate rain,0.1,98.0,4.0,1012.0
12306,10234289,R-e3a6b473,2019-02-12 07:00:00,2019-02-12 21:00:00+00:00,0,2019-02-12 21:00:00+00:00,2019-02-12 07:00:00+00:00,C-c7cacd1d,C-b6e04c88,705.50,...,1021.0,C-b6e04c88,2019-02-12 21:00:00+00:00,28.0,16.0,Clear,0.0,61.0,6.0,1020.0


In [208]:
# Function to ensure datetime columns are in UTC
def ensure_utc(column):
    if column.dt.tz is None:  # Check if the column is naive
        return column.dt.tz_localize('UTC')  # Localize to UTC
    else:
        return column.dt.tz_convert('UTC')  # Convert to UTC if already aware

# Ensure both datetime columns are in UTC
nearest_hour_schedule_df['departure_date'] = ensure_utc(nearest_hour_schedule_df['departure_date'])
nearest_hour_schedule_df['estimated_arrival'] = ensure_utc(nearest_hour_schedule_df['estimated_arrival'])

# Now create the custom date ranges
hourly_exploded_scheduled_df = (
    nearest_hour_schedule_df.assign(
        custom_date=[
            pd.date_range(start, end, freq='H')  # Create custom date ranges
            for start, end in zip(
                nearest_hour_schedule_df['departure_date'],
                nearest_hour_schedule_df['estimated_arrival']
            )  # Using departure and estimated arrival times
        ]
    ).explode('custom_date', ignore_index=True)  # Explode the DataFrame based on the custom date range
)

# Display the resulting DataFrame
print(hourly_exploded_scheduled_df[['departure_date', 'estimated_arrival', 'custom_date']].head())


             departure_date         estimated_arrival  \
0 2019-02-09 07:00:00+00:00 2019-02-10 03:00:00+00:00   
1 2019-02-09 07:00:00+00:00 2019-02-10 03:00:00+00:00   
2 2019-02-09 07:00:00+00:00 2019-02-10 03:00:00+00:00   
3 2019-02-09 07:00:00+00:00 2019-02-10 03:00:00+00:00   
4 2019-02-09 07:00:00+00:00 2019-02-10 03:00:00+00:00   

                custom_date  
0 2019-02-09 07:00:00+00:00  
1 2019-02-09 08:00:00+00:00  
2 2019-02-09 09:00:00+00:00  
3 2019-02-09 10:00:00+00:00  
4 2019-02-09 11:00:00+00:00  


In [209]:
traffic_table

,route_id,date,no_of_vehicles,accident
0,R-c970877c,2019-02-10 05:00:00+00:00,573.0,0
1,R-2ef954d3,2019-02-09 03:00:00+00:00,609.0,0
2,R-3d32a465,2019-01-28 21:00:00+00:00,159.0,0
3,R-071c5bc9,2019-02-05 10:00:00+00:00,2577.0,0
4,R-5a83ad98,2019-01-07 15:00:00+00:00,2563.0,0
...,...,...,...,...
2597908,R-47aae3c4,2019-01-12 17:00:00+00:00,2457.0,0
2597909,R-2f8939a6,2019-02-05 20:00:00+00:00,580.0,0
2597910,R-3b99b000,2019-01-29 06:00:00+00:00,567.0,0
2597911,R-a49eac13,2019-01-20 12:00:00+00:00,144.0,0


In [211]:
hourly_exploded_scheduled_df

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,city_id_y,custom_date_y,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y,custom_date
0,87511111,R-c0901e00,2019-02-09 07:00:00+00:00,2019-02-10 03:00:00+00:00,0,2019-02-10 03:00:00+00:00,2019-02-09 07:00:00+00:00,C-9bbf5d8a,C-2c9e75ef,1016.34,...,C-2c9e75ef,2019-02-10 03:00:00+00:00,34.0,6.0,Overcast,0.0,89.0,6.0,1025.0,2019-02-09 07:00:00+00:00
1,87511111,R-c0901e00,2019-02-09 07:00:00+00:00,2019-02-10 03:00:00+00:00,0,2019-02-10 03:00:00+00:00,2019-02-09 07:00:00+00:00,C-9bbf5d8a,C-2c9e75ef,1016.34,...,C-2c9e75ef,2019-02-10 03:00:00+00:00,34.0,6.0,Overcast,0.0,89.0,6.0,1025.0,2019-02-09 08:00:00+00:00
2,87511111,R-c0901e00,2019-02-09 07:00:00+00:00,2019-02-10 03:00:00+00:00,0,2019-02-10 03:00:00+00:00,2019-02-09 07:00:00+00:00,C-9bbf5d8a,C-2c9e75ef,1016.34,...,C-2c9e75ef,2019-02-10 03:00:00+00:00,34.0,6.0,Overcast,0.0,89.0,6.0,1025.0,2019-02-09 09:00:00+00:00
3,87511111,R-c0901e00,2019-02-09 07:00:00+00:00,2019-02-10 03:00:00+00:00,0,2019-02-10 03:00:00+00:00,2019-02-09 07:00:00+00:00,C-9bbf5d8a,C-2c9e75ef,1016.34,...,C-2c9e75ef,2019-02-10 03:00:00+00:00,34.0,6.0,Overcast,0.0,89.0,6.0,1025.0,2019-02-09 10:00:00+00:00
4,87511111,R-c0901e00,2019-02-09 07:00:00+00:00,2019-02-10 03:00:00+00:00,0,2019-02-10 03:00:00+00:00,2019-02-09 07:00:00+00:00,C-9bbf5d8a,C-2c9e75ef,1016.34,...,C-2c9e75ef,2019-02-10 03:00:00+00:00,34.0,6.0,Overcast,0.0,89.0,6.0,1025.0,2019-02-09 11:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311568,64713507,R-42c1d264,2019-01-09 07:00:00+00:00,2019-01-13 09:00:00+00:00,1,2019-01-13 09:00:00+00:00,2019-01-09 07:00:00+00:00,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-13 05:00:00+00:00
311569,64713507,R-42c1d264,2019-01-09 07:00:00+00:00,2019-01-13 09:00:00+00:00,1,2019-01-13 09:00:00+00:00,2019-01-09 07:00:00+00:00,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-13 06:00:00+00:00
311570,64713507,R-42c1d264,2019-01-09 07:00:00+00:00,2019-01-13 09:00:00+00:00,1,2019-01-13 09:00:00+00:00,2019-01-09 07:00:00+00:00,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-13 07:00:00+00:00
311571,64713507,R-42c1d264,2019-01-09 07:00:00+00:00,2019-01-13 09:00:00+00:00,1,2019-01-13 09:00:00+00:00,2019-01-09 07:00:00+00:00,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-13 08:00:00+00:00


In [212]:
# Merge the traffic_table and hourly_exploded_scheduled_df DataFrames
scheduled_traffic = hourly_exploded_scheduled_df.merge(
    traffic_table,
    left_on=['route_id', 'custom_date'],  # Columns from hourly_exploded_scheduled_df
    right_on=['route_id', 'date'],        # Corresponding columns from traffic_table
    how='left'
)

# Display the resulting merged DataFrame
print(scheduled_traffic.head())


   truck_id    route_id            departure_date         estimated_arrival  \
0  87511111  R-c0901e00 2019-02-09 07:00:00+00:00 2019-02-10 03:00:00+00:00   
1  87511111  R-c0901e00 2019-02-09 07:00:00+00:00 2019-02-10 03:00:00+00:00   
2  87511111  R-c0901e00 2019-02-09 07:00:00+00:00 2019-02-10 03:00:00+00:00   
3  87511111  R-c0901e00 2019-02-09 07:00:00+00:00 2019-02-10 03:00:00+00:00   
4  87511111  R-c0901e00 2019-02-09 07:00:00+00:00 2019-02-10 03:00:00+00:00   

   delay estimated_arrival_nearest_hour departure_date_nearest_hour  \
0      0      2019-02-10 03:00:00+00:00   2019-02-09 07:00:00+00:00   
1      0      2019-02-10 03:00:00+00:00   2019-02-09 07:00:00+00:00   
2      0      2019-02-10 03:00:00+00:00   2019-02-09 07:00:00+00:00   
3      0      2019-02-10 03:00:00+00:00   2019-02-09 07:00:00+00:00   
4      0      2019-02-10 03:00:00+00:00   2019-02-09 07:00:00+00:00   

    origin_id destination_id  distance  ...  wind_speed_y description_y  \
0  C-9bbf5d8a     C-2c9

In [213]:
#Define a custom aggregation function for accidents
def custom_agg(values):
    """
    Custom aggregation function to determine if any value in a group is 1 (indicating an accident).
    
    Args:
    values (iterable): Iterable of values in a group.
    
    Returns:
    int: 1 if any value is 1, else 0.
    """
    # Check if any value in the iterable is equal to 1
    return 1 if 1 in values else 0

# Group by 'unique_id', 'truck_id', and 'route_id', and apply custom aggregation
scheduled_route_traffic = scheduled_traffic.groupby([ 'truck_id', 'route_id'], as_index=False).agg(
    avg_no_of_vehicles=('no_of_vehicles', 'mean'),
    accident=('accident', custom_agg)
)

# Display the resulting DataFrame
print(scheduled_route_traffic.head())

   truck_id    route_id  avg_no_of_vehicles  accident
0  10008392  R-06dfe39e         1877.916667         0
1  10008392  R-35dca729         1820.098039         0
2  10008392  R-51d1a32a         1536.542373         0
3  10008392  R-7ab5ae85         1927.825000         0
4  10008392  R-c6f8e036          829.283333         0


Merge Data

In [214]:
#renaming the schedule_data_merge with origin_destination_weather
origin_destination_weather=schedule_data_merge

In [215]:
#Merge schedule_route_traffic with origin_destination_weather

origin_destination_weather_traffic_merge=origin_destination_weather.merge(scheduled_route_traffic,on=['truck_id','route_id'],how='left')

merge weather &traffic

In [217]:
# Convert 'departure_date' to datetime in the schedule_weather_merge DataFrame
schedule_weather_merge['departure_date'] = pd.to_datetime(schedule_weather_merge['departure_date'], errors='coerce')

# If origin_destination_weather_traffic_merge has 'departure_date' as object, convert it as well
origin_destination_weather_traffic_merge['departure_date'] = pd.to_datetime(origin_destination_weather_traffic_merge['departure_date'], errors='coerce')


In [218]:
#merged_data_weather_traffic=pd.merge(schedule_weather_merge, origin_destination_weather_traffic_merge, on=['truck_id', 'route_id', 'departure_date','estimated_arrival', 'delay'], how='left')

# Merge the DataFrames
merged_data_weather_traffic = pd.merge(
    schedule_weather_merge, 
    origin_destination_weather_traffic_merge, 
    on=['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay'], 
    how='left'
)

# Display the merged DataFrame
print(merged_data_weather_traffic.head())


   truck_id    route_id      departure_date         estimated_arrival  delay  \
0  87511111  R-c0901e00 2019-02-09 07:00:00 2019-02-10 03:00:00+00:00      0   
1  18910102  R-8eabc8ee 2019-01-07 07:00:00 2019-01-08 02:00:00+00:00      1   
2  10946672  R-1ec2684e 2019-01-25 07:00:00 2019-01-26 02:00:00+00:00      0   
3  16497961  R-2e2bd615 2019-02-05 07:00:00 2019-02-06 07:00:00+00:00      1   
4  18882093  R-1acdeb76 2019-01-01 07:00:00 2019-01-02 04:00:00+00:00      0   

   route_avg_temp  route_avg_wind_speed  route_avg_precip  route_avg_humidity  \
0            58.6              4.400000               0.0           87.600000   
1            46.0              8.000000               0.0           97.333333   
2            74.0              4.600000               0.0           43.800000   
3            60.0              4.666667               0.0           94.000000   
4            48.4              3.600000               0.0           70.200000   

   route_avg_visibility  ...    

merge weather traffic trucks

In [221]:
merged_data_weather_traffic_trucks = pd.merge(merged_data_weather_traffic, trucks_table, on='truck_id', how='left')
merged_data_weather_traffic_trucks

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,...,precip_y,humidity_y,visibility_y,pressure_y,avg_no_of_vehicles,accident,truck_age,load_capacity_pounds,mileage_mpg,fuel_type
0,87511111,R-c0901e00,2019-02-09 07:00:00,2019-02-10 03:00:00+00:00,0,58.600000,4.400000,0.0,87.600000,5.000000,...,0.0,89.0,6.0,1025.0,1890.142857,1,11.0,3000.0,22.0,diesel
1,18910102,R-8eabc8ee,2019-01-07 07:00:00,2019-01-08 02:00:00+00:00,1,46.000000,8.000000,0.0,97.333333,4.333333,...,0.0,95.0,1.0,1008.0,1999.400000,0,9.0,4000.0,17.0,gas
2,10946672,R-1ec2684e,2019-01-25 07:00:00,2019-01-26 02:00:00+00:00,0,74.000000,4.600000,0.0,43.800000,6.000000,...,0.0,75.0,6.0,1027.0,1973.650000,0,11.0,20000.0,29.0,diesel
3,16497961,R-2e2bd615,2019-02-05 07:00:00,2019-02-06 07:00:00+00:00,1,60.000000,4.666667,0.0,94.000000,2.666667,...,0.0,89.0,3.0,1022.0,1487.640000,0,12.0,6000.0,17.0,gas
4,18882093,R-1acdeb76,2019-01-01 07:00:00,2019-01-02 04:00:00+00:00,0,48.400000,3.600000,0.0,70.200000,6.000000,...,0.0,77.0,6.0,1027.0,1965.318182,0,10.0,15000.0,21.0,diesel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12303,95354743,R-6d4858a1,2019-01-07 07:00:00,2019-01-08 00:00:00+00:00,0,55.000000,5.000000,0.1,96.500000,4.000000,...,0.0,79.0,6.0,1011.0,2139.444444,0,10.0,4000.0,17.0,gas
12304,57362582,R-1d7b758f,2019-01-10 07:00:00,2019-01-10 08:00:00+00:00,0,65.000000,8.000000,0.0,82.000000,6.000000,...,0.0,60.0,6.0,1000.0,2175.500000,0,9.0,6000.0,21.0,diesel
12305,22376424,R-c165d4fd,2019-02-06 07:00:00,2019-02-06 08:00:00+00:00,0,59.000000,8.000000,0.0,64.500000,6.000000,...,0.1,98.0,4.0,1012.0,2599.000000,0,7.0,6000.0,12.0,gas
12306,10234289,R-e3a6b473,2019-02-12 07:00:00,2019-02-12 21:00:00+00:00,0,59.750000,5.000000,0.0,84.250000,6.000000,...,0.0,61.0,6.0,1020.0,1452.466667,0,8.0,15000.0,13.0,gas


Merge merged_data with truck_data based on 'truck_id' column (Left Join)

In [222]:
final_merge = pd.merge(merged_data_weather_traffic_trucks, drivers_table, left_on='truck_id', right_on = 'vehicle_no', how='left')

Function to check if there is nighttime involved between arrival and departure time

In [223]:
def has_midnight(start, end):
       return int(start.date() != end.date())

Apply the function to create a new column indicating nighttime involvement

In [224]:
final_merge['is_midnight'] = final_merge.apply(lambda row: has_midnight(row['departure_date'], row['estimated_arrival']), axis=1)

In [225]:
final_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,87511111,R-c0901e00,2019-02-09 07:00:00,2019-02-10 03:00:00+00:00,0,58.600000,4.400000,0.0,87.600000,5.000000,...,2e4f9185-4,Cole Krueger,male,46.0,19.0,conservative,6.0,87511111.0,49.38,1
1,18910102,R-8eabc8ee,2019-01-07 07:00:00,2019-01-08 02:00:00+00:00,1,46.000000,8.000000,0.0,97.333333,4.333333,...,1aadd95f-1,David Frye,male,50.0,10.0,conservative,8.0,18910102.0,41.03,1
2,10946672,R-1ec2684e,2019-01-25 07:00:00,2019-01-26 02:00:00+00:00,0,74.000000,4.600000,0.0,43.800000,6.000000,...,b10fcfc9-e,Tony Robbins,male,42.0,16.0,proactive,6.0,10946672.0,61.28,1
3,16497961,R-2e2bd615,2019-02-05 07:00:00,2019-02-06 07:00:00+00:00,1,60.000000,4.666667,0.0,94.000000,2.666667,...,d80a340b-d,Matthew Erickson,male,47.0,9.0,proactive,8.0,16497961.0,58.89,1
4,18882093,R-1acdeb76,2019-01-01 07:00:00,2019-01-02 04:00:00+00:00,0,48.400000,3.600000,0.0,70.200000,6.000000,...,effe4a4d-0,Michael Jacobs,male,47.0,10.0,proactive,8.0,18882093.0,61.46,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12303,95354743,R-6d4858a1,2019-01-07 07:00:00,2019-01-08 00:00:00+00:00,0,55.000000,5.000000,0.1,96.500000,4.000000,...,63acb645-b,Andrew Wells,male,49.0,15.0,proactive,5.0,95354743.0,59.01,1
12304,57362582,R-1d7b758f,2019-01-10 07:00:00,2019-01-10 08:00:00+00:00,0,65.000000,8.000000,0.0,82.000000,6.000000,...,72b3c3a0-e,Evan Jackson,male,54.0,21.0,proactive,7.0,57362582.0,58.05,0
12305,22376424,R-c165d4fd,2019-02-06 07:00:00,2019-02-06 08:00:00+00:00,0,59.000000,8.000000,0.0,64.500000,6.000000,...,f4a568c6-0,Noah Shah,male,39.0,3.0,conservative,3.0,22376424.0,47.85,0
12306,10234289,R-e3a6b473,2019-02-12 07:00:00,2019-02-12 21:00:00+00:00,0,59.750000,5.000000,0.0,84.250000,6.000000,...,5050d6b3-b,Maurice Howe,male,53.0,26.0,proactive,6.0,10234289.0,61.97,0


Dropping the duplicates

In [228]:
final_merge = final_merge.drop_duplicates()
final_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,87511111,R-c0901e00,2019-02-09 07:00:00,2019-02-10 03:00:00+00:00,0,58.600000,4.400000,0.0,87.600000,5.000000,...,2e4f9185-4,Cole Krueger,male,46.0,19.0,conservative,6.0,87511111.0,49.38,1
1,18910102,R-8eabc8ee,2019-01-07 07:00:00,2019-01-08 02:00:00+00:00,1,46.000000,8.000000,0.0,97.333333,4.333333,...,1aadd95f-1,David Frye,male,50.0,10.0,conservative,8.0,18910102.0,41.03,1
2,10946672,R-1ec2684e,2019-01-25 07:00:00,2019-01-26 02:00:00+00:00,0,74.000000,4.600000,0.0,43.800000,6.000000,...,b10fcfc9-e,Tony Robbins,male,42.0,16.0,proactive,6.0,10946672.0,61.28,1
3,16497961,R-2e2bd615,2019-02-05 07:00:00,2019-02-06 07:00:00+00:00,1,60.000000,4.666667,0.0,94.000000,2.666667,...,d80a340b-d,Matthew Erickson,male,47.0,9.0,proactive,8.0,16497961.0,58.89,1
4,18882093,R-1acdeb76,2019-01-01 07:00:00,2019-01-02 04:00:00+00:00,0,48.400000,3.600000,0.0,70.200000,6.000000,...,effe4a4d-0,Michael Jacobs,male,47.0,10.0,proactive,8.0,18882093.0,61.46,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12303,95354743,R-6d4858a1,2019-01-07 07:00:00,2019-01-08 00:00:00+00:00,0,55.000000,5.000000,0.1,96.500000,4.000000,...,63acb645-b,Andrew Wells,male,49.0,15.0,proactive,5.0,95354743.0,59.01,1
12304,57362582,R-1d7b758f,2019-01-10 07:00:00,2019-01-10 08:00:00+00:00,0,65.000000,8.000000,0.0,82.000000,6.000000,...,72b3c3a0-e,Evan Jackson,male,54.0,21.0,proactive,7.0,57362582.0,58.05,0
12305,22376424,R-c165d4fd,2019-02-06 07:00:00,2019-02-06 08:00:00+00:00,0,59.000000,8.000000,0.0,64.500000,6.000000,...,f4a568c6-0,Noah Shah,male,39.0,3.0,conservative,3.0,22376424.0,47.85,0
12306,10234289,R-e3a6b473,2019-02-12 07:00:00,2019-02-12 21:00:00+00:00,0,59.750000,5.000000,0.0,84.250000,6.000000,...,5050d6b3-b,Maurice Howe,male,53.0,26.0,proactive,6.0,10234289.0,61.97,0


In [ ]:
final_merge = final_merge.drop(columns=['city_id_x','city_id_y','custom_date_x'])
final_merge

In [236]:
final_merge=final_merge.drop(columns=['custom_date_y'])

In [237]:
final_merge.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'route_avg_temp', 'route_avg_wind_speed', 'route_avg_precip',
       'route_avg_humidity', 'route_avg_visibility', 'route_avg_pressure',
       'route_description', 'estimated_arrival_nearest_hour',
       'departure_date_nearest_hour', 'origin_id', 'destination_id',
       'distance', 'average_hours', 'temp_x', 'wind_speed_x', 'description_x',
       'precip_x', 'humidity_x', 'visibility_x', 'pressure_x', 'temp_y',
       'wind_speed_y', 'description_y', 'precip_y', 'humidity_y',
       'visibility_y', 'pressure_y', 'avg_no_of_vehicles', 'accident',
       'truck_age', 'load_capacity_pounds', 'mileage_mpg', 'fuel_type',
       'driver_id', 'name', 'gender', 'age', 'experience', 'driving_style',
       'ratings', 'vehicle_no', 'average_speed_mph', 'is_midnight'],
      dtype='object')

In [238]:
final_merge.shape

(12308, 48)

In [239]:
final_merge['unique_id'] = final_merge.index

In [240]:
final_merge.shape

(12308, 49)

In [243]:
# Get the count of NaN values in each column
final_merge.isnull().sum()

truck_id                             0
route_id                             0
departure_date                       0
estimated_arrival                    0
delay                                0
route_avg_temp                      43
route_avg_wind_speed                43
route_avg_precip                    43
route_avg_humidity                  43
route_avg_visibility                43
route_avg_pressure                  43
route_description                   43
estimated_arrival_nearest_hour       0
departure_date_nearest_hour          0
origin_id                          859
destination_id                     859
distance                           859
average_hours                      859
temp_x                            1411
wind_speed_x                      1411
description_x                     1411
precip_x                          1411
humidity_x                        1411
visibility_x                      1411
pressure_x                        1411
temp_y                   

In [244]:
# Drop rows with NaN values
final_merge.dropna(inplace=True)

In [245]:
final_merge.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
route_avg_temp                    0
route_avg_wind_speed              0
route_avg_precip                  0
route_avg_humidity                0
route_avg_visibility              0
route_avg_pressure                0
route_description                 0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
origin_id                         0
destination_id                    0
distance                          0
average_hours                     0
temp_x                            0
wind_speed_x                      0
description_x                     0
precip_x                          0
humidity_x                        0
visibility_x                      0
pressure_x                        0
temp_y                            0
wind_speed_y                      0
description_y               

In [246]:
final_merge.shape

(10472, 49)